# Dependencies

In [1]:
!pip install gdown
!pip install bnunicodenormalizer
!pip install pytorch-pretrained-bert
# git clone model
!git clone https://github.com/imranulashrafi/banner.git
# copy model to models
!cp /kaggle/input/gpun-ner-model/models/14.pt banner/models/
# download dataset
!git clone https://github.com/MISabic/NER-Bangla-Dataset.git
# unzip 
!unzip NER-Bangla-Dataset/Bangla-NER-Splitted-Dataset.zip -d dataset 
# download weight-file
!gdown 1LtE0By2_cHXoHP0is0l0DVwfXIHBHXEV
# move model to models
!mv banner_model.pt banner/models/banner_model.pt

  Installing build dependencies ... - \ | / - \ done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
  Created wheel for gdown: filename=gdown-4.5.3-py3-none-any.whl size=14821 sha256=425f613918f62f714dda711ac8c187dbcce911e12100e01b74bc96255dd34fef
  Stored in directory: /root/.cache/pip/wheels/94/8d/0b/bdcd83555c3555f91a33f6c2384428d9f163c7d75ab0d272b4
Successfully built gdown
  Preparing metadata (setup.py) ... - done
  Created wheel for bnunicodenormalizer: filename=bnunicodenormalizer-0.1.0-py3-none-any.whl size=32742 sha256=3954f4832a20840e9446976acde4ed34169441533aebefd42004955ab9a57c40
  Stored in directory: /root/.cache/pip/wheels/ed/e2/1b/2ceac3443a89e3869e118a197dd02c99921dddbcccacf9474c
Successfully built bnunicodenormalizer
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 605.5 kB/s eta 0:00:00
Cloning into 'banner'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), d

# Dataset

In [2]:
#-------------------------
# imports
#-------------------------
import sys
sys.path.append("./banner")
import json
import os
import pandas as pd 
from tqdm import tqdm
from bnunicodenormalizer import Normalizer 
from sklearn.metrics import classification_report
from net import Net
from dataset import NerDataset, pad, VOCAB
from trainer import eval
import torch
import numpy as np
import random

tqdm.pandas()
# initialize
bnorm=Normalizer()

#-------------------------
# format raw test set
#-------------------------
with open("./dataset/Bangla-NER-Splitted-Dataset.json","r") as f:
    dataset=json.load(f)



100%|██████████| 995526/995526 [00:00<00:00, 2284583.97B/s]


In [3]:

def infect(word, random_number = 75):
    # ref: https://colab.research.google.com/drive/17cP5xTdMf_hahluDyBr0h70j2gxNMiAb?usp=sharing
    nukta_map              =   {'য়':'য়',
                            'র':'ব়',
                            'ড়':'ড়',
                            'ঢ়':'ঢ়'}
    diacritic_map           =   {'ো':'ো',
                                'ৌ':'ৌ',
                                'অা':'আ',
                                'ৃ':'ৄ'}


    vowel_diacritics       =   ['া', 'ি', 'ী', 'ু', 'ূ', 'ৃ', 'ে', 'ৈ', 'ো', 'ৌ']

    non_gylph_unicodes     =   ['\u0984', '\u098d','\u098e','\u0991','\u0992','\u09a9','\u09b1','\u09b3','\u09b4','\u09b5',
                                    '\u09ba','\u09bb', '\u09c5','\u09c6','\u09c9','\u09ca','\u09cf','\u09d0','\u09d1','\u09d2',
                                    '\u09d3','\u09d4','\u09d5','\u09d6', '\u09d8','\u09d9','\u09da','\u09db','\u09de', '\u09e4',
                                    '\u09e5','\u09ff']

    vowels                 =   ['অ', 'আ', 'ই', 'ঈ', 'উ', 'ঊ', 'ঋ', 'এ', 'ঐ', 'ও', 'ঔ']

    consonants             =   ['ক', 'খ', 'গ', 'ঘ', 'ঙ', 'চ', 'ছ','জ', 'ঝ', 'ঞ', 'ট', 'ঠ', 'ড', 'ঢ', 'ণ', 'ত', 'থ', 'দ', 'ধ', 'ন', 
                                    'প', 'ফ', 'ব', 'ভ', 'ম', 'য', 'র', 'ল', 'শ', 'ষ', 'স', 'হ','ড়', 'ঢ়', 'য়','ৎ']


    connector              =   '্'
    nukta                  =   '়'
    original="".join([c for c in word])
    try:
    
        random_number = random_number % 5


        part = [i for i in word]
        new_word = []

        if random.randint(0, 100) > 70:
            index_connector = random.randint(1, len(word))
            word = word[:index_connector] + non_gylph_unicodes[random.randint(0, len(non_gylph_unicodes))] + word[index_connector:]
            index_connector = random.randint(1, len(word))
            word = word[:index_connector] + connector + word[index_connector:]

        for i in word:
            if i in vowels:
                if random.randint(0, 100) > 50:
                    word = word.replace(i, i+vowel_diacritics[random.randint(0, len(vowel_diacritics))])

        for i in part:
            for k in list(nukta_map):
                if i == k:
                    word = word.replace(k, nukta_map[k])

        for i in range(len(part)):
            for k in list(diacritic_map):
                if part[i] == k:
                    d_part = [_ for _ in diacritic_map[k]]
                    word = ("".join(part[:i]) + d_part[0] + d_part[1] + "".join(part[i+1:]))
                    #word = "".join(part)

        return word
    except Exception as e:
        return original


In [4]:
def norm_sentence(sent):
    words=[]
    for word in sent:
        data=bnorm(word)
        if data["normalized"] is None:
            words.append(word)
        else:
            words.append(data["normalized"])
    return words

* original --> df
* normalized--> dfn
* infected1-->df1
* infected2-->df2
* infected5-->df5
* infected1 norm-->df1n
* infected2 norm-->df2n
* infected5 norm-->df5n


In [5]:
# original
df=pd.DataFrame(dataset["test"])
# normalied
dfn=df.copy() 
dfn.sentence=dfn.sentence.progress_apply(lambda x:norm_sentence(x))

# infected-1
df1=df.copy() 
df1.sentence=df1.sentence.progress_apply(lambda x:[infect(i) for i in x])

# infected-2
df2=df1.copy() 
df2.sentence=df2.sentence.progress_apply(lambda x:[infect(i) for i in x])
# infected-5
df5=df.copy()
for i in range(5):
    df5.sentence=df5.sentence.progress_apply(lambda x:[infect(i) for i in x])

# infected-1-norm
df1n=df1.copy() 
df1n.sentence=df1n.sentence.progress_apply(lambda x:norm_sentence(x))
# infected-2-norm
df2n=df2.copy() 
df2n.sentence=df2n.sentence.progress_apply(lambda x:norm_sentence(x))
# infected-5-norm
df5n=df5.copy() 
df5n.sentence=df5n.sentence.progress_apply(lambda x:norm_sentence(x))


100%|██████████| 3564/3564 [00:19<00:00, 178.33it/s]


# Load models

In [6]:
# model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
top_rnns=True
model_s = Net(top_rnns, len(VOCAB), device, finetuning=True)
model_b = Net(top_rnns, len(VOCAB), device, finetuning=True)
# trained with normalized data here: https://www.kaggle.com/code/nazmuddhohaansary/gpun-ner-train-with-normalized-data
model_s.load_state_dict(torch.load('banner/models/14.pt'))
model_s=model_s.to(device)
# zero - shot
model_b.load_state_dict(torch.load('banner/models/banner_model.pt'))
model_b=model_b.to(device)

def run_ner_infer(sent,model):
    tags = []
    for x in range(len(sent.split())):
        tags.append('O')
    sent_infer=[]
    sent_infer.append(["[CLS]"] + sent.split() + ["[SEP]"])
    tags_infer=[]
    tags_infer.append(["<PAD>"] + tags + ["<PAD>"])

    infer_data = NerDataset(sent_infer, tags_infer)

    infer_iter = torch.utils.data.DataLoader(dataset=infer_data,
                             batch_size=1,
                             shuffle=False,
                             collate_fn = pad,
                             num_workers=0
                             )
    pred = eval(model, infer_iter)
    for x in range(len(pred[0])):
        if pred[0][x] == '<PAD>':
            pred[0][x] = 'O'
    return pred[0][1:-1]

100%|██████████| 662804195/662804195 [00:24<00:00, 27584966.88B/s]


In [7]:
def eval_data(df):
    preds=[]
    gts=[]
    for idx in tqdm(range(len(df))):
        pred=df.iloc[idx,-1]
        gt=df.iloc[idx,-2]
        preds+=pred
        gts+=gt
    eval_predictions(preds,gts)

def eval_predictions(pred,gt):
    pred=[VOCAB.index(p) for p in tqdm(pred)]
    gt=[VOCAB.index(p) for p in tqdm(gt)]
    report = classification_report(np.array(gt),np.array(pred), labels = [1,2,3,4,5,6,7,8,9], digits=4, zero_division=0)
    print(report)    



In [8]:
def get_stat(model,df):
    df["predicted_tags"]=df.sentence.progress_apply(lambda x:run_ner_infer(" ".join(x),model))
    eval_data(df)

# Exp-1: Original

In [9]:
print("banner - original")
print("-------------------------------------")
get_stat(model_b,df)
print("scratch - original")
print("-------------------------------------")
get_stat(model_s,df)

banner - original
-------------------------------------


100%|██████████| 49118/49118 [00:00<00:00, 1786176.61it/s]


              precision    recall  f1-score   support

           1     0.6973    0.6583    0.6772       357
           2     0.5186    0.5569    0.5370       853
           3     0.9559    0.9463    0.9511     39499
           4     0.3778    0.2982    0.3333        57
           5     0.7919    0.7874    0.7897      1223
           6     0.4578    0.4921    0.4743       573
           7     0.5152    0.6589    0.5782       516
           8     0.7879    0.7706    0.7791      2005
           9     0.7616    0.8045    0.7825      4035

    accuracy                         0.9056     49118
   macro avg     0.6515    0.6637    0.6558     49118
weighted avg     0.9084    0.9056    0.9068     49118

scratch - original
-------------------------------------


100%|██████████| 49118/49118 [00:00<00:00, 1982217.45it/s]


              precision    recall  f1-score   support

           1     0.7218    0.5742    0.6396       357
           2     0.5957    0.4560    0.5166       853
           3     0.9515    0.9513    0.9514     39499
           4     0.7778    0.1228    0.2121        57
           5     0.7764    0.8005    0.7882      1223
           6     0.5153    0.4119    0.4578       573
           7     0.5649    0.5310    0.5475       516
           8     0.7970    0.7441    0.7697      2005
           9     0.7356    0.8392    0.7840      4035

    accuracy                         0.9069     49118
   macro avg     0.7151    0.6035    0.6297     49118
weighted avg     0.9059    0.9069    0.9055     49118



# Exp-2 Normalized

In [10]:
print("banner - normalized")
print("-------------------------------------")
get_stat(model_b,dfn)
print("scratch - normalized")
print("-------------------------------------")
get_stat(model_s,dfn)

banner - normalized
-------------------------------------


100%|██████████| 49118/49118 [00:00<00:00, 1624973.96it/s]


              precision    recall  f1-score   support

           1     0.7057    0.5574    0.6228       357
           2     0.4687    0.5264    0.4959       853
           3     0.9525    0.9408    0.9466     39499
           4     0.3529    0.3158    0.3333        57
           5     0.7737    0.7686    0.7711      1223
           6     0.4206    0.4764    0.4468       573
           7     0.4736    0.6434    0.5456       516
           8     0.7752    0.7257    0.7496      2005
           9     0.7378    0.7993    0.7673      4035

    accuracy                         0.8969     49118
   macro avg     0.6290    0.6393    0.6310     49118
weighted avg     0.9010    0.8969    0.8985     49118

scratch - normalized
-------------------------------------


100%|██████████| 49118/49118 [00:00<00:00, 1268297.00it/s]


              precision    recall  f1-score   support

           1     0.7128    0.5910    0.6462       357
           2     0.6065    0.4572    0.5214       853
           3     0.9494    0.9522    0.9508     39499
           4     0.5385    0.1228    0.2000        57
           5     0.7702    0.7890    0.7795      1223
           6     0.5261    0.4049    0.4576       573
           7     0.5705    0.5329    0.5511       516
           8     0.7929    0.7197    0.7545      2005
           9     0.7341    0.8287    0.7786      4035

    accuracy                         0.9055     49118
   macro avg     0.6890    0.5998    0.6266     49118
weighted avg     0.9038    0.9055    0.9038     49118



# Exp-3 Unnorm-1

In [11]:
print("banner - un-normalized-1")
print("-------------------------------------")
get_stat(model_b,df1)
print("scratch - un-normalized-1")
print("-------------------------------------")
get_stat(model_s,df1)
print("scratch - normalized-1")
print("-------------------------------------")
get_stat(model_s,df1n)

banner - un-normalized-1
-------------------------------------


100%|██████████| 49118/49118 [00:00<00:00, 1958213.64it/s]


              precision    recall  f1-score   support

           1     0.5125    0.1148    0.1876       357
           2     0.5088    0.2708    0.3535       853
           3     0.8815    0.9646    0.9212     39499
           4     0.4000    0.0351    0.0645        57
           5     0.6507    0.5102    0.5720      1223
           6     0.4167    0.1222    0.1889       573
           7     0.5073    0.3372    0.4051       516
           8     0.6599    0.3591    0.4651      2005
           9     0.6348    0.4394    0.5193      4035

    accuracy                         0.8497     49118
   macro avg     0.5747    0.3504    0.4086     49118
weighted avg     0.8274    0.8497    0.8307     49118

scratch - un-normalized-1
-------------------------------------


100%|██████████| 49118/49118 [00:00<00:00, 1881955.84it/s]


              precision    recall  f1-score   support

           1     0.5957    0.0784    0.1386       357
           2     0.5974    0.2122    0.3131       853
           3     0.8920    0.9513    0.9207     39499
           4     0.4000    0.0351    0.0645        57
           5     0.5411    0.6141    0.5753      1223
           6     0.4080    0.1239    0.1901       573
           7     0.5105    0.2364    0.3232       516
           8     0.6912    0.3516    0.4661      2005
           9     0.5772    0.5460    0.5611      4035

    accuracy                         0.8477     49118
   macro avg     0.5792    0.3499    0.3947     49118
weighted avg     0.8317    0.8477    0.8320     49118

scratch - normalized-1
-------------------------------------


100%|██████████| 49118/49118 [00:00<00:00, 1912707.61it/s]


              precision    recall  f1-score   support

           1     0.6380    0.5826    0.6091       357
           2     0.6023    0.4279    0.5003       853
           3     0.9476    0.9478    0.9477     39499
           4     0.5385    0.1228    0.2000        57
           5     0.7502    0.7809    0.7652      1223
           6     0.5147    0.3979    0.4488       573
           7     0.5509    0.5136    0.5316       516
           8     0.7686    0.7072    0.7366      2005
           9     0.7185    0.8230    0.7672      4035

    accuracy                         0.9000     49118
   macro avg     0.6699    0.5893    0.6118     49118
weighted avg     0.8986    0.9000    0.8984     49118



# Exp-4 Unnorm-2

In [12]:
print("banner - un-normalized-2")
print("-------------------------------------")
get_stat(model_b,df2)
print("scratch - un-normalized-2")
print("-------------------------------------")
get_stat(model_s,df2)
print("scratch - normalized-2")
print("-------------------------------------")
get_stat(model_s,df2n)

banner - un-normalized-2
-------------------------------------


100%|██████████| 49118/49118 [00:00<00:00, 1954071.69it/s]


              precision    recall  f1-score   support

           1     0.5000    0.0616    0.1097       357
           2     0.5149    0.2227    0.3110       853
           3     0.8625    0.9700    0.9131     39499
           4     0.0000    0.0000    0.0000        57
           5     0.6036    0.3835    0.4690      1223
           6     0.3670    0.0698    0.1173       573
           7     0.4893    0.2655    0.3442       516
           8     0.6509    0.2753    0.3870      2005
           9     0.5941    0.3341    0.4277      4035

    accuracy                         0.8362     49118
   macro avg     0.5091    0.2869    0.3421     49118
weighted avg     0.8060    0.8362    0.8081     49118

scratch - un-normalized-2
-------------------------------------


100%|██████████| 49118/49118 [00:00<00:00, 1951350.91it/s]


              precision    recall  f1-score   support

           1     0.5385    0.0392    0.0731       357
           2     0.6207    0.1688    0.2654       853
           3     0.8793    0.9528    0.9146     39499
           4     0.0000    0.0000    0.0000        57
           5     0.5045    0.5511    0.5268      1223
           6     0.4286    0.0890    0.1474       573
           7     0.5661    0.2074    0.3035       516
           8     0.6626    0.2703    0.3840      2005
           9     0.5391    0.4803    0.5080      4035

    accuracy                         0.8369     49118
   macro avg     0.5266    0.3065    0.3470     49118
weighted avg     0.8166    0.8369    0.8160     49118

scratch - normalized-2
-------------------------------------


100%|██████████| 49118/49118 [00:00<00:00, 1826705.30it/s]


              precision    recall  f1-score   support

           1     0.5941    0.5658    0.5796       357
           2     0.6062    0.4115    0.4902       853
           3     0.9462    0.9457    0.9459     39499
           4     0.5000    0.1228    0.1972        57
           5     0.7387    0.7743    0.7561      1223
           6     0.5045    0.3944    0.4427       573
           7     0.5389    0.5097    0.5239       516
           8     0.7555    0.6888    0.7206      2005
           9     0.7074    0.8171    0.7583      4035

    accuracy                         0.8964     49118
   macro avg     0.6546    0.5811    0.6016     49118
weighted avg     0.8952    0.8964    0.8949     49118



# Exp-5 Unnorm 5

In [13]:
print("banner - un-normalized-5")
print("-------------------------------------")
get_stat(model_b,df5)
print("scratch - un-normalized-5")
print("-------------------------------------")
get_stat(model_s,df5)
print("scratch - normalized-5")
print("-------------------------------------")
get_stat(model_s,df5n)

banner - un-normalized-5
-------------------------------------


100%|██████████| 49118/49118 [00:00<00:00, 2073762.12it/s]


              precision    recall  f1-score   support

           1     0.4000    0.0112    0.0218       357
           2     0.4461    0.1067    0.1722       853
           3     0.8312    0.9832    0.9008     39499
           4     0.0000    0.0000    0.0000        57
           5     0.5680    0.1570    0.2460      1223
           6     0.3111    0.0244    0.0453       573
           7     0.3534    0.0911    0.1448       516
           8     0.6259    0.1352    0.2223      2005
           9     0.5567    0.1703    0.2608      4035

    accuracy                         0.8172     49118
   macro avg     0.4547    0.1866    0.2238     49118
weighted avg     0.7719    0.8172    0.7662     49118

scratch - un-normalized-5
-------------------------------------


100%|██████████| 49118/49118 [00:00<00:00, 1938862.97it/s]


              precision    recall  f1-score   support

           1     0.5000    0.0168    0.0325       357
           2     0.6321    0.0785    0.1397       853
           3     0.8527    0.9598    0.9031     39499
           4     0.0000    0.0000    0.0000        57
           5     0.4130    0.3532    0.3808      1223
           6     0.4138    0.0209    0.0399       573
           7     0.6393    0.0756    0.1352       516
           8     0.6339    0.1287    0.2139      2005
           9     0.4449    0.3304    0.3792      4035

    accuracy                         0.8156     49118
   macro avg     0.5033    0.2182    0.2471     49118
weighted avg     0.7846    0.8156    0.7802     49118

scratch - normalized-5
-------------------------------------


100%|██████████| 49118/49118 [00:00<00:00, 1947532.44it/s]


              precision    recall  f1-score   support

           1     0.6182    0.5714    0.5939       357
           2     0.5848    0.3962    0.4724       853
           3     0.9432    0.9425    0.9428     39499
           4     0.6667    0.1404    0.2319        57
           5     0.7103    0.7637    0.7360      1223
           6     0.4819    0.3717    0.4197       573
           7     0.5364    0.5000    0.5176       516
           8     0.7524    0.6683    0.7079      2005
           9     0.6946    0.8112    0.7484      4035

    accuracy                         0.8916     49118
   macro avg     0.6654    0.5739    0.5967     49118
weighted avg     0.8906    0.8916    0.8900     49118

